In [ ]:
from dotenv import load_dotenv

# 토큰 정보 로드
load_dotenv()

In [ ]:
import json

from langchain_core.messages import (
    BaseMessage,
    FunctionMessage,
    HumanMessage,
)

from langchain.tools.render import format_tool_to_openai_function
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import END, StateGraph
from langgraph.prebuilt.tool_executor import ToolExecutor, ToolInvocation


def create_agent(llm, tools, system_message: str):
    # 에이전트를 생성합니다.
    functions = [format_tool_to_openai_function(t) for t in tools]

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful AI assistant, collaborating with other assistants."
                " Use the provided tools to progress towards answering the question."
                " If you are unable to fully answer, that's OK, another assistant with different tools "
                " will help where you left off. Execute what you can to make progress."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    prompt = prompt.partial(tool_names=", ".join(
        [tool.name for tool in tools]))
    return prompt | llm.bind_functions(functions)

In [ ]:
from langchain_core.tools import tool
from typing import Annotated
from langchain_experimental.utilities import PythonREPL
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results=5)

repl = PythonREPL()


@tool
def python_repl(
    code: Annotated[str, "The python code to execute to generate your chart."]
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    return f"Succesfully executed:\n```python\n{code}\n```\nStdout: {result}"

In [ ]:
import operator
from typing import Annotated, Sequence, Tuple, TypedDict, Union
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict


# 각 에이전트와 도구에 대한 다른 노드를 생성할 것입니다. 이 클래스는 그래프의 각 노드 사이에서 전달되는 객체를 정의합니다.
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str

In [ ]:
import functools


def agent_node(state, agent, name):
    result = agent.invoke(state)
    if isinstance(result, FunctionMessage):
        pass
    else:
        result = HumanMessage(**result.dict(exclude={"type", "name"}), name=name)
    return {
        "messages": [result],
        "sender": name,
    }


llm = ChatOpenAI(model="gpt-4-1106-preview")

# Research agent and node
research_agent = create_agent(
    llm,
    [tavily_tool],
    system_message="You should provide accurate data for the chart generator to use.",
)
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

# Chart Generator
chart_agent = create_agent(
    llm,
    [python_repl],
    system_message="Any charts you display will be visible by the user.",
)
chart_node = functools.partial(agent_node, agent=chart_agent, name="Chart Generator")

In [ ]:
tools = [tavily_tool, python_repl]
tool_executor = ToolExecutor(tools)


def tool_node(state):
    # 그래프에서 도구를 실행하는 함수입니다.
    # 에이전트 액션을 입력받아 해당 도구를 호출하고 결과를 반환합니다.
    messages = state["messages"]
    # 계속 조건에 따라 마지막 메시지가 함수 호출을 포함하고 있음을 알 수 있습니다.
    last_message = messages[-1]
    # ToolInvocation을 함수 호출로부터 구성합니다.
    tool_input = json.loads(
        last_message.additional_kwargs["function_call"]["arguments"]
    )
    # 단일 인자 입력은 값으로 직접 전달할 수 있습니다.
    if len(tool_input) == 1 and "__arg1" in tool_input:
        tool_input = next(iter(tool_input.values()))
    tool_name = last_message.additional_kwargs["function_call"]["name"]
    action = ToolInvocation(
        tool=tool_name,
        tool_input=tool_input,
    )
    # 도구 실행자를 호출하고 응답을 받습니다.
    response = tool_executor.invoke(action)
    # 응답을 사용하여 FunctionMessage를 생성합니다.
    function_message = FunctionMessage(
        content=f"{tool_name} response: {str(response)}", name=action.tool
    )
    # 기존 리스트에 추가될 리스트를 반환합니다.
    return {"messages": [function_message]}

In [ ]:
def router(state):
    # 상태 정보를 기반으로 다음 단계를 결정하는 라우터 함수
    messages = state["messages"]
    last_message = messages[-1]
    if "function_call" in last_message.additional_kwargs:
        # 이전 에이전트가 도구를 호출함
        return "call_tool"
    if "FINAL ANSWER" in last_message.content:
        # 어느 에이전트든 작업이 끝났다고 결정함
        return "end"
    return "continue"

In [ ]:
workflow = StateGraph(AgentState)

workflow.add_node("Researcher", research_node)
workflow.add_node("Chart Generator", chart_node)
workflow.add_node("call_tool", tool_node)

workflow.add_conditional_edges(
    "Researcher",
    router,
    {"continue": "Chart Generator", "call_tool": "call_tool", "end": END},
)
workflow.add_conditional_edges(
    "Chart Generator",
    router,
    {"continue": "Researcher", "call_tool": "call_tool", "end": END},
)

workflow.add_conditional_edges(
    "call_tool",
    lambda x: x["sender"],
    {
        "Researcher": "Researcher",
        "Chart Generator": "Chart Generator",
    },
)
workflow.set_entry_point("Researcher")
graph = workflow.compile()

In [16]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(
                content="대한민국의 2018년 ~ 2022년의 합계출산율에 대한 데이터를 찾아줘, "
                "그리고 2023, 2024, 2025년에 대한 전망도 찾아줘, "
                "마지막으로 수집한 데이터에 대한 그래프를 그려줘. "
                "코드 작성을 완료했다면 종료해줘."
            )
        ],
    },
    # 그래프에서 수행할 최대 단계 수
    {"recursion_limit": 200},
):
    print(s)
    print("----")

{'Researcher': {'messages': [HumanMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"NVIDIA monthly stock prices January 2020 to October 2023"}', 'name': 'tavily_search_results_json'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 265, 'total_tokens': 296}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_992d1ea92d', 'finish_reason': 'function_call', 'logprobs': None}, name='Researcher', id='run-7dbf8bfe-d1f3-4343-8ae3-76c61dcb6148-0', tool_calls=[], usage_metadata={'input_tokens': 265, 'output_tokens': 31, 'total_tokens': 296}, invalid_tool_calls=[])], 'sender': 'Researcher'}}
----


/var/folders/hx/s86zllf119jg8r80vywnjxnh0000gn/T/ipykernel_46310/1130803573.py:19: LangGraphDeprecationWarning: ToolInvocation is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  action = ToolInvocation(


{'call_tool': {'messages': [FunctionMessage(content="tavily_search_results_json response: [{'url': 'https://ycharts.com/companies/NVDA/ytd_monthly_return', 'content': 'In depth view into NVIDIA Year to Date Price Returns (Monthly) including historical data from 1999, charts and stats. ... October 31, 2023-- September 30, 2023 ... October 31, 2020--'}, {'url': 'https://www.statmuse.com/money/ask/nvidia-stock-price-january-2023', 'content': '401,276,596. Daily pricing data for NVIDIA dates back to 1/22/1999, and may be incomplete. The closing price for NVIDIA (NVDA) in January 2023 was $19.53, on January 31, 2023. It was up 31.6% for the month. The latest price is $129.37.'}, {'url': 'https://www.nasdaq.com/market-activity/stocks/nvda/historical', 'content': 'RUA. Russell 3000. 3,099.15. -9.82 -0.32%. Find the latest historical data for NVIDIA Corporation Common Stock (NVDA) at Nasdaq.com. View historical data in a monthly, bi-annual, or yearly format.'}, {'url': 'https://au.finance.yaho

/var/folders/hx/s86zllf119jg8r80vywnjxnh0000gn/T/ipykernel_46310/1130803573.py:19: LangGraphDeprecationWarning: ToolInvocation is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  action = ToolInvocation(


{'call_tool': {'messages': [FunctionMessage(content='tavily_search_results_json response: [{\'url\': \'https://www.benzinga.com/money/nvidia-stock-price-prediction\', \'content\': \'Nvidia Stock Price Prediction: 2024, 2025, 2030 - Benzinga Nvidia Stock Price Prediction: 2024, 2025, 2030 Nvidia Stock Price Prediction: 2024, 2025, 2030 But what does the future hold for Nvidia’s stock price? Current Overview of Nvidia Stock Nvidia Stock Price Prediction for 2024 Nvidia Stock Price Prediction for 2025 Nvidia Stock Price Prediction for 2030 How much is Nvidia stock? Currently, Nvidia’s stock is trading around $130. Why is Nvidia stock so high? Nvidia’s dominant position in AI and data centers and strong revenue growth have driven its stock price to record levels. How to buy Nvidia stock?\'}, {\'url\': \'https://stockscan.io/stocks/NVDA/forecast\', \'content\': \'NVIDIA Corp Stock Price Forecast 2024, 2025, 2030 to 2050\'}, {\'url\': \'https://markets.businessinsider.com/news/stocks/nvidia-